In [ ]:
import torch
import pandas as pd
from PIL import Image
from transformers import CLIPProcessor, CLIPModel
from tqdm import tqdm
import os
import pickle

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(f"🚀 Creando el catálogo definitivo en: {DEVICE}")

# 1. CARGAMOS EXACTAMENTE EL MISMO MODELO QUE USAMOS PARA INFERENCIA
model_id = "openai/clip-vit-base-patch32"
processor = CLIPProcessor.from_pretrained(model_id)
model = CLIPModel.from_pretrained(model_id).to(DEVICE)

# 2. RUTAS
RUTA_FOTOS_PRODUCTOS = "../data/images/products/" # ⚠️ Asegúrate de que esta es la ruta a las fotos del catálogo
RUTA_CSV_PRODUCTOS = "../data/raw/product_train.csv" # ⚠️ El CSV que tiene la info de productos
RUTA_NUEVO_PKL = "../data/embeddings/catalog_embeddings_v2_seguro.pkl"

df_productos = pd.read_csv(RUTA_CSV_PRODUCTOS)
catalogo_db = {}

print(f"⚙️ Procesando {len(df_productos)} productos...")

# 3. BUCLE DE EXTRACCIÓN
model.eval()
with torch.no_grad():
    for _, fila in tqdm(df_productos.iterrows(), total=len(df_productos)):
        id_producto = fila['product_asset_id']
        ruta_foto = os.path.join(RUTA_FOTOS_PRODUCTOS, f"{id_producto}.jpg")
        
        if os.path.exists(ruta_foto):
            try:
                # Abrir imagen
                imagen = Image.open(ruta_foto).convert("RGB")
                
                # Pasar por CLIP
                inputs = processor(images=imagen, return_tensors="pt").to(DEVICE)
                vision_outputs = model.vision_model(pixel_values=inputs['pixel_values'])
                vector = model.visual_projection(vision_outputs.pooler_output)
                
                # Normalizar MUY IMPORTANTE
                vector = vector / vector.norm(dim=-1, keepdim=True)
                
                # Guardar en diccionario (quitando la dimensión extra y pasándolo a CPU)
                catalogo_db[id_producto] = vector.cpu().squeeze().numpy()
                
            except Exception as e:
                print(f"Error con {id_producto}: {e}")

# 4. GUARDAR EL NUEVO PKL
os.makedirs(os.path.dirname(RUTA_NUEVO_PKL), exist_ok=True)
with open(RUTA_NUEVO_PKL, "wb") as f:
    pickle.dump(catalogo_db, f)

print(f"\n✅ ¡Catálogo V2 generado con éxito! Guardado en: {RUTA_NUEVO_PKL}")